# Python U-Net 网络图像分割实验 - 以眼底 OCT 图像为例

> Copyright (c) 2023 Vincent SHI | 史文朔

本实验是基于 Python 语言的 U-Net 网络图像分割实验，以眼底 OCT 图像为例，实现了 U-Net 网络的训练、测试等功能，并以此对 Python 语言的基本语法、图像处理等知识进行了综合性的实践。

本实验依赖眼底 OCT 图像数据集，建议同学们通过参加相关竞赛自行获取。


## 实验目的

通过本实验，你必须要掌握：

- Python 语言的基本语法
- Python 语言的基本图像处理
  - 图像读取
  - 图像显示
  - 图像标注
  - 图像保存
- 诸如 U-Net 等深度学习网络的基本定义
- Python 机器学习生态库的基本调用

同时，建议你掌握：

- Python 语言的基本科学计算库
- Python 语言的基本深度学习库
- U-Net 等深度学习网络的基本训练方法
- U-Net 等深度学习网络的基本原理

你可以先不必掌握：

- Python 图像处理库的内部实现
- U-Net 等深度学习网络的数学理论基础


## 实验步骤

- 引用相关库

- 图片的读取和预处理

- 划分训练和测试集

- 定义 U-Net 网络

- 进行训练和测试

- 对验证集进行预测

- 输出预测结果


## 实验代码样例

### 引用相关库


### 图片的读取和预处理


### 划分训练和测试集


### 定义 U-Net 网络


### 进行训练和测试


### 对验证集进行预测


### 输出预测结果
